<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angel Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: FELIPE GALAZ GARCIA
- Nombre de alumno 2: MIRKO MARTINIC JARA


### **Link de repositorio de GitHub:** `https://github.com/mmartinicj/MDS7202-labs`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [18]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

# Librería para omitir FutureWarnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [19]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito. 
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [20]:
df_retail = pd.read_pickle("Lab_07/online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos] 

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente. 

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.
 

$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Respuesta:**

In [21]:
from datetime import timedelta

def custom_features(dataframe_in):
    
    df = dataframe_in.copy()
    
    # generamos atributo Length
    date_ini = df.groupby(by=['Customer ID']).agg({'InvoiceDate':'min'})
    date_fin = df.groupby(by=['Customer ID']).agg({'InvoiceDate':'max'})
    length = pd.DataFrame((date_fin - date_ini)['InvoiceDate'].dt.days).rename(columns={'InvoiceDate':'Length'})

    # generamos atributo Recency
    date_ini = df.groupby(by=['Customer ID']).agg({'InvoiceDate':'max'})
    date_fin = df.InvoiceDate.max() + timedelta(days=1)
    recency = pd.DataFrame((date_fin - date_ini)['InvoiceDate'].dt.days).rename(columns={'InvoiceDate':'Recency'})

    # generamos atributo Monetary
    monetary = df.groupby(by=['Customer ID']).agg({'Price':'mean'}).rename(columns={'Price':'Monetary'}).round(2)
    
    # generamos atributo Frequency
    # SUPUESTO : SE CONSIDERA QUE UNA VISITA REPRESENTA LAS COMPRAS REALIZADAS EN UNA HORA DADA
    frequency = df.groupby(by=['Customer ID']).agg({'InvoiceDate':'count'}).rename(columns={'InvoiceDate':'Frequency'})
    
    # generamos atributo Periodicity
    # SUPUESTO : SE CONSIDERA QUE UNA VISITA REPRESENTA LAS COMPRAS REALIZADAS EN UNA HORA DADA
    df = df.sort_values(by=['Customer ID', 'InvoiceDate']).drop_duplicates('InvoiceDate')
    df['Periodicity'] = np.where(df['Customer ID']==df['Customer ID'].shift(1), 
                                 df.InvoiceDate.diff().dt.days, 
                                 np.nan)
    periodicity = df.groupby('Customer ID').agg({'Periodicity':'std'}).round(1).fillna(0)
    
    # creamos dataframe con los atributos
    dataframe_out = pd.concat([length, recency, monetary, frequency, periodicity], axis=1)
    dataframe_out.insert(0, 'Customer ID', dataframe_out.index)
    
    return dataframe_out.reset_index(drop=True)

In [22]:
custom_features(df_retail).head()

,Customer ID,Length,Recency,Monetary,Frequency,Periodicity
0,12346.0,196,165,6.25,33,36.7
1,12347.0,37,3,2.30,71,0.0
2,12348.0,0,74,0.72,20,0.0
3,12349.0,181,43,8.58,102,101.8
4,12351.0,0,11,2.36,21,0.0


## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [23]:
class MinMax(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return (X - X.min()) / (X.max() - X.min())

### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo. 

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. [X] Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. [X] En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior. 
3. [X] Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [24]:
# creamos pipeline segun indicaciones
pipe = Pipeline([('trf1', FunctionTransformer(custom_features)),
                 ('trf2', ColumnTransformer([('scale', MinMax(), ['Length', 'Recency', 'Monetary', 'Frequency', 'Periodicity']
                                             )], remainder='drop')), # SE DESCARTA EL ATRIBUTO Customer ID PARA APLICAR TSNE
                 ('trf3', TSNE(n_components=2, random_state=0))])

# aplicamos transformaciones
rates_tsne = pipe.fit(df_retail).fit_transform(df_retail)

# visualizamos componentes obtenidas
fig = px.scatter(x=rates_tsne[:,0], y=rates_tsne[:,1])
fig.update_layout(
    title="tSNE",
    xaxis_title="1ra Componente",
    yaxis_title="2da Componente")
fig.show()

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [25]:
# lista vacia para almacenar SSE por cluster
SSE = []

# ejecutamos pipeline para cada cluster en el conjunto [1,20]
for k in range(1, 21): 
    pipe = Pipeline([('trf1', FunctionTransformer(custom_features)),
                     ('trf2', ColumnTransformer([('scale', MinMax(), ['Length', 'Recency', 'Monetary', 'Frequency', 'Periodicity']
                                                 )], remainder='drop')), # remainder='passthrough' 
                     ('trf3', KMeans(n_clusters=k, n_init='auto', random_state=0))])
    pipe.fit(df_retail)
    SSE.append(pipe.named_steps['trf3'].inertia_)
    
# graficamos SSE vs. n_clusters 
fig = px.line(x=np.arange(1,21), y=SSE, markers=True) 
fig.update_layout(
    title="Método del Codo",
    xaxis_title="Número de Clusters",
    yaxis_title="SSE")
fig.show()

De acuerdo con la gráfica anterior, al evaluar la suma de la diferencia al cuadrado entre los puntos de cada cluster (SSE, por sus siglas en inglés) en función del número de clusters, se reconoce que el codo se ubicaría en `n_clusters=4`, por lo cual, se selecciona dicho valor para implementar, en la siguiente sección, el algoritmo K-Means.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [26]:
# creamos dataframe para almacenar informacion de clusters
clusters = custom_features(df_retail)

# creamos pipeline estableciendo la cantidad de clusters optima definida
pipe = Pipeline([('trf1', FunctionTransformer(custom_features)),
                 ('trf2', ColumnTransformer([('scale', MinMax(), ['Length', 'Recency', 'Monetary', 'Frequency', 'Periodicity']
                                             )], remainder='drop')), # remainder='passthrough' 
                 ('trf3', KMeans(n_clusters=4, n_init='auto', random_state=0))])

# aplicamos transformaciones y agregamos etiquetas obtenidas al dataframe
pipe.fit(df_retail).transform(df_retail)
clusters['Cluster'] = pipe.named_steps['trf3'].labels_

# generamos resumen de valores medios en atributos por cluster
summary = clusters.groupby('Cluster').mean(numeric_only=True).round(2)
summary['Count'] = clusters.groupby('Cluster').count().Length
summary

,Length,Recency,Monetary,Frequency,Periodicity,Count
Cluster,,,,,,
0,18.00,255.20,27.00,26.53,1.95,914
1,319.23,23.67,3.60,210.85,45.66,1111
2,17.64,52.25,8.09,39.73,2.00,1283
3,182.96,66.73,3.46,90.92,29.93,1006


En virtud de la tabla anterior, se observan agrupaciones coherentes que sustentan la definición de los siguientes tipos de clientes:

- **Grupo 1** (`cluster=0`). Clientes poco fieles, con bajo interés en repetir sus compras en alguno de los locales y alta contribución en los ingresos de la empresa


- **Grupo 2** (`cluster=1`). Clientes altamente fieles, con alto interés en repetir sus compras en alguno de los locales y baja contribución en los ingresos de la empresa


- **Grupo 3** (`cluster=2`). Clientes poco fieles, con alto interés en repetir sus compras en alguno de los locales y baja contribución en los ingresos de la empresa


- **Grupo 4** (`cluster=3`). Clientes medianamente fieles, con alto interés en repetir sus compras en alguno de los locales y baja contribución en los ingresos de la empresa

Si consideramos la segmentación de Peker *et al.* (2017), entonces las agrupaciones pueden ser renombradas como:

- **Grupo 1** (`cluster=0`) = **High-spending lost customers**


- **Grupo 2** (`cluster=1`) = **Low-contribution loyal customers**


- **Grupo 3** (`cluster=2`) = **Low-spending lost customers**


- **Grupo 4** (`cluster=3`) = **Uncertain customers**


---
Peker, S., Kocyigit, A. & Eren, P. (2017). *LRFMP model for customer segmentation in the grocery retail industry: a case study*.

**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [27]:
# visualizamos componentes obtenidas coloreadas segun clusters generados
fig = px.scatter(x=rates_tsne[:,0], y=rates_tsne[:,1], color=clusters.Cluster.astype('str'))
fig.update_layout(
    title="tSNE",
    xaxis_title="1ra Componente",
    yaxis_title="2da Componente",
    legend_title='Grupo de Cliente'
)
fig.show()

Según la visualización anterior, se reconoce que se separan adecuadamente los distintos clusters generados vía K-Means, por lo que se puede suponer válida la descripción de los grupos de clientes expuesta en la sección precedente.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>